In [37]:
from keras.models import load_model
import pandas as pd
import numpy as np
import os
import re

In [38]:
model = load_model('ner_elmo_bilstm.h5')

In [39]:
from allennlp.modules.elmo import Elmo,batch_to_ids

In [40]:
# sentence = "उत्तराखंड की राजधानी देहरादून में फरवरी की दुपहरी पिछले दस साल में सबसे गर्म रही".split()
# sentence = "अरे सारथी जम्मू कैसा मौसम है".split()
sentence = "मुझे बताओ हांगकांग में तापमान क्या है".split()

In [41]:
X = [w for w in sentence]
X

['मुझे', 'बताओ', 'हांगकांग', 'में', 'तापमान', 'क्या', 'है']

In [42]:
max_len = 16

In [43]:
if len(X)<max_len:
    while len(X)<max_len:
        X += ["__PAD__"]
X = [X]

In [44]:
datadir = os.path.join('hi', 'elmo')
options_file = os.path.join(datadir, 'hi-d512-options.json')
weight_file = os.path.join(datadir, 'hi-d512-elmo.hdf5')

elmo = Elmo(options_file,weight_file,1,dropout=0)

In [45]:
embeds = elmo(batch_to_ids(X))
embedding = embeds["elmo_representations"]

In [46]:
import torch
import tensorflow as tf

embedding1 = []
for t in embedding:
    np_tensor = t.detach().numpy()
    embedding1 += [np_tensor.tolist()]

embedding = embedding1

In [47]:
# embedding

In [48]:
p = model.predict(np.array(embedding).reshape(1,16,1024))
p = np.argmax(p, axis=-1)

In [49]:
p

array([[6, 6, 9, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]], dtype=int64)

In [50]:
df = pd.DataFrame(columns=['Sentence #','Word','Tag'])

In [51]:
data_dir = r'Data/'

In [52]:
i=1
data = list()
with open(data_dir+'nertrnweather.txt',encoding='utf-8') as f:
    for line in f.readlines():
        if line=='\n':
            i+=1
        else:
            data=line.split(" ")
            df=df.append({"Sentence #":f"Sentence {i}","Word":data[0],"Tag":re.sub("\n","",data[1])},ignore_index=True)
df

,Sentence #,Word,Tag
0,Sentence 1,अरे,O
1,Sentence 1,सारथी,O
2,Sentence 1,जम्मू,U-location
3,Sentence 1,कैसा,O
4,Sentence 1,मौसम,O
...,...,...,...
1403,Sentence 203,की,O
1404,Sentence 203,स्थिति,O
1405,Sentence 203,क्या,O
1406,Sentence 203,है,O


In [53]:
tags = list(set(df["Tag"].values))
n_tags = len(tags); n_tags

10

In [54]:
p.tolist()

[[6, 6, 9, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]]

In [57]:
for w,pred in zip(X[0],p.tolist()[0]):
        print(w,tags[pred])

मुझे B-date
बताओ B-date
हांगकांग L-date
में B-date
तापमान B-date
क्या B-date
है B-date
__PAD__ B-date
__PAD__ B-date
__PAD__ B-date
__PAD__ B-date
__PAD__ B-date
__PAD__ B-date
__PAD__ B-date
__PAD__ B-date
__PAD__ B-date


In [56]:
tags

['U-weather_type',
 'O',
 'U-date',
 'L-weather_type',
 'B-location',
 'L-location',
 'B-date',
 'U-location',
 'B-weather_type',
 'L-date']